In [780]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Viterbi

In [782]:
# From "https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm"
def viterbi(y, A, B, Pi=None):
    """
    Return the MAP estimate of state trajectory of Hidden Markov Model.

    Parameters
    ----------
    y : array (T,)
        Observation state sequence. int dtype.
    A : array (K, K)
        State transition matrix. See HiddenMarkovModel.state_transition  for
        details.
    B : array (K, M)
        Emission matrix. See HiddenMarkovModel.emission for details.
    Pi: optional, (K,)
        Initial state probabilities: Pi[i] is the probability x[0] == i. If
        None, uniform initial distribution is assumed (Pi[:] == 1/K).

    Returns
    -------
    x : array (T,)
        Maximum a posteriori probability estimate of hidden state trajectory,
        conditioned on observation sequence y under the model parameters A, B,
        Pi.
    T1: array (K, T)
        the probability of the most likely path so far
    T2: array (K, T)
        the x_j-1 of the most likely path so far
    """
    
    # Cardinality of the state space
    K = A.shape[0]
    # Initialize the priors with default (uniform dist) if not given by caller
    Pi = Pi if Pi is not None else np.full(K, 1 / K)
    T = len(y)
    T1 = np.empty((K, T), 'd')
    T2 = np.empty((K, T), 'B')

    # Initialize the tracking tables from first observation
    T1[:, 0] = np.log(Pi * B[:, y[0]])
    T2[:, 0] = 0

    # Iterate through the observations updating the tracking tables
    for i in range(1, T):
        #need to subtract 1 from y as it ranges from 1-6 but indexed 0 in array
        T1[:, i] = np.max(T1[:, i - 1] + np.log(A), 1) + np.log(B[np.newaxis, :, y[i] - 1])
        T2[:, i] = np.argmax(T1[:, i - 1] + np.log(A), 1)

    # Build the output, optimal model trajectory
    x = np.empty(T, 'B')
    x[-1] = np.argmax(T1[:, T - 1])
    #flip since we are backtracking
    for i in reversed(range(1, T)):
        x[i - 1] = T2[x[i], i]

    return x, T1, T2

In [783]:
obs_a = np.loadtxt("../datasets/markov/hmm_pb1.csv", dtype = int, delimiter = ",")
obs_a_pd = pd.read_csv("../datasets/markov/hmm_pb1.csv", header=None)
pi_a = np.array([0.5, 0.5])
trans_a = np.array([
                    [0.95, 0.05],
                    [0.05, 0.95]
                   ])
emit_a = np.array([
                    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6],
                    [1/10, 1/10, 1/10, 1/10, 1/10, 1/2]
                  ])

In [1186]:
y, T1, T2 = viterbi(obs_a, trans_a, emit_a, pi_a)

In [1188]:
#y = 1 is Fair, y=2 is loaded
print(y+1)
print(T1)
print(T2)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[  -2.48490665   -4.32795941   -6.17101218   -8.01406494   -9.8571177
   -11.70017047  -13.54322323  -15.386276    -17.22932876  -19.07238152
   -20.91543429  -22.75848705  -24.60153981  -26.44459258  -28.28764534
   -30.1306981   -31.97375087  -33.81680363  -35.65985639  -37.50290916
   -39.34596192  -41.18901469  -43.03206745  -44.87512021  -46.71817298
   -48.56122574  -50.4042785   -52.24733127  -54.09038403  -55.93343679
   -57.776

# Forward Algorithm

In [1216]:
#Modified from "http://www.adeveloperdiary.com/data-science/machine-learning/forward-and-backward-algorithm-in-hidden-markov-model/"
def forward(observations, transition_matrix, emission_matrix, pi):
    #python indexed 0 so subtract 1 from all dice rolls
    observations = observations - 1
    #get number of observations
    observations_rows = observations.shape[0]
    #get number of states (K)
    K = transition_matrix.shape[0]
    
    #Initialize alpha probabilities
    alpha = np.zeros([observations_rows, K])
    alpha[0, :] = pi * emission_matrix[:, observations[0]]
    alpha_probs = np.zeros([observations_rows, K])
    alpha_probs[0] = alpha[0] / np.sum(alpha[0])
    
    #common factor to avoid underflow
    #u = np.zeros([observations_rows - 1, K])
    
    #iterate forward through observations to compute alpha probabilities
    for t, obs in enumerate(observations[1:], 1):
            alpha[t] = np.array([
                            #start with probability we go from previous state to fair
                            emission_matrix[0][obs] * np.sum([
                                #chance from fair to fair
                                alpha[t - 1][0] * transition_matrix[0, 0],
                                #chance from loaded to fair
                                alpha[t - 1][1] * transition_matrix[1, 0]
                                ]),
                            #go from previous state to loaded
                            emission_matrix[1][obs] * np.sum([
                                #chance from fair to loaded
                                alpha[t - 1][0] * transition_matrix[0, 1],
                                #chance from loaded to loaded
                                alpha[t - 1][0] * transition_matrix[1, 1]
                                ]),
                            ])
            alpha_probs[t] = np.array([
                            #start with probability we go from previous state to fair
                            emission_matrix[0][obs] * np.sum([
                                #chance from fair to fair
                                alpha_probs[t - 1][0] * transition_matrix[0, 0],
                                #chance from loaded to fair
                                alpha_probs[t - 1][1] * transition_matrix[1, 0]
                                ]),
                            #go from previous state to loaded
                            emission_matrix[1][obs] * np.sum([
                                #chance from fair to loaded
                                alpha_probs[t - 1][0] * transition_matrix[0, 1],
                                #chance from loaded to loaded
                                alpha_probs[t - 1][0] * transition_matrix[1, 1]
                                ]),
                            ])
            #normalize for probabilities
            alpha_probs[t] /= np.sum(alpha_probs[t])
            #still need u_t for underflow ?
            #u[t - 1, :] = alpha[t] / alpha[t - 1]
            
    print(alpha[128][0] / alpha[128][1])
    return alpha_probs

In [1214]:
alpha = forward(obs_a, trans_a, emit_a, pi_a)

1.6343228459730739


In [1215]:
print(alpha)

[[0.625      0.375     ]
 [0.62025316 0.37974684]
 [0.62040026 0.37959974]
 [0.62039567 0.37960433]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.24634358 0.75365642]
 [0.64767406 0.35232594]
 [0.61957884 0.38042116]
 [0.62042133 0.37957867]
 [0.24634295 0.75365705]
 [0.26882218 0.73117782]
 [0.64412739 0.35587261]
 [0.61968134 0.38031866]
 [0.62041812 0.37958188]
 [0.62039511 0.37960489]
 [0.62039583 0.37960417]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.24634358 0.75365642]
 [0.64767406 0.35232594]
 [0.61957884 0.38042116]
 [0.62042133 0.37957867]
 [0.62039501 0.37960499]
 [0.62039583 0.37960417]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.62039581 0.37960419]
 [0.24634358 0.75365642]
 [0.26882208 0.73117792]
 [0.6441274  0.3558726 ]


# Backwards Algorithm

In [1217]:
#Modified from from "http://www.adeveloperdiary.com/data-science/machine-learning/forward-and-backward-algorithm-in-hidden-markov-model/"
def backward(observations, transition_matrix, emission_matrix):
    #python indexed 0 so subtract 1 from all dice rolls
    observations = observations - 1
    #get number of observations
    observations_rows = observations.shape[0]
    #get number of states
    K = transition_matrix.shape[0]
    
    #initial probabilities for beta start at T and work backwards
    beta, beta_probs = np.zeros([observations_rows, K]), np.zeros([observations_rows, K])
    beta[observations_rows - 1] = np.array([1, 1])
    beta_probs[observations_rows - 1] = np.array([1, 1])
    
    for t, obs in reversed(list(enumerate(observations))[1:]):
        beta[t - 1] = np.array([
            np.sum([
                #from fair to fair
                transition_matrix[0, 0] * emission_matrix[0][obs] * beta[t][0],
                #from fair to loaded
                transition_matrix[0, 1] * emission_matrix[1][obs] * beta[t][1],
            ]),
            np.sum([
                #from loaded to fair
                transition_matrix[1, 0] * emission_matrix[0][obs] * beta[t][0],
                #from loaded to loaded
                transition_matrix[1, 1] * emission_matrix[1][obs] * beta[t][1]
            ])
        ])
        beta_probs[t - 1] = np.array([
            np.sum([
                #from fair to fair
                transition_matrix[0, 0] * emission_matrix[0][obs] * beta_probs[t][0],
                #from fair to loaded
                transition_matrix[0, 1] * emission_matrix[1][obs] * beta_probs[t][1],
            ]),
            np.sum([
                #from loaded to fair
                transition_matrix[1, 0] * emission_matrix[0][obs] * beta_probs[t][0],
                #from loaded to loaded
                transition_matrix[1, 1] * emission_matrix[1][obs] * beta_probs[t][1]
            ])
        ])
        
        #normalize beta values
        beta_probs[t - 1] /= np.sum(beta_probs[t - 1])
    print(beta[128][0] / beta[128][1])
    
    return beta_probs

In [1218]:
beta = backward(obs_a, trans_a, emit_a)

0.8001349068941145


In [1219]:
print(beta)

[[0.83753763 0.16246237]
 [0.80775169 0.19224831]
 [0.76168793 0.23831207]
 [0.69396316 0.30603684]
 [0.60144619 0.39855381]
 [0.48698444 0.51301556]
 [0.73899454 0.26100546]
 [0.66206804 0.33793196]
 [0.56052515 0.43947485]
 [0.44024075 0.55975925]
 [0.69665882 0.30334118]
 [0.88449408 0.11550592]
 [0.88430652 0.11569348]
 [0.88399063 0.11600937]
 [0.8834588  0.1165412 ]
 [0.88256396 0.11743604]
 [0.88105982 0.11894018]
 [0.87853579 0.12146421]
 [0.87431232 0.12568768]
 [0.86727865 0.13272135]
 [0.85565695 0.14434305]
 [0.83670249 0.16329751]
 [0.80643459 0.19356541]
 [0.75969346 0.24030654]
 [0.6911223  0.3088777 ]
 [0.88137091 0.11862909]
 [0.87905737 0.12094263]
 [0.87518386 0.12481614]
 [0.86872667 0.13127333]
 [0.85804017 0.14195983]
 [0.84056451 0.15943549]
 [0.81253797 0.18746203]
 [0.76896549 0.23103451]
 [0.70439185 0.29560815]
 [0.61518097 0.38481903]
 [0.5031825  0.4968175 ]
 [0.37832098 0.62167902]
 [0.63274854 0.36725146]
 [0.84640431 0.15359569]
 [0.82182856 0.17817144]


# 2. Baum Welch Algorithm

In [958]:
obs_b = pd.read_csv("../datasets/markov/hmm_pb2.csv", header=None).values
obs_b = obs_b[0]
trans_EM = np.random.rand(2, 2)
trans_EM /= trans_EM.sum(axis=1)[:, None]
emit_EM = np.random.rand(2, 6)
emit_EM /= emit_EM.sum(axis=1)[:, None]
pi_EM = np.random.rand(2)
pi_EM /= pi_EM.sum()

#Check initial guesses
print(trans_EM)
print(emit_EM)
print(pi_EM)
#Check their probabilities add to 1
print(trans_EM.sum(axis=1))
print(emit_EM.sum(axis=1))
print(np.sum(pi_EM))

[[0.794 0.206]
 [0.776 0.224]]
[[0.149 0.271 0.02  0.267 0.119 0.175]
 [0.171 0.13  0.2   0.131 0.218 0.149]]
[0.478 0.522]
[1. 1.]
[1. 1.]
0.9999999999999999


In [1221]:
def baum_welch(observations, transition_matrix, emission_matrix, pi, epochs = 50):
    
    observations = observations - 1
    observations_rows = observations.shape[0]
    #number of states
    K = transition_matrix.shape[0]
    
    for epoch in range(epochs):
        #initialize placeholders for observations
        new_transition_matrix = np.zeros((2, 2))
        new_pi = np.zeros(2)
        new_emission_matrix = np.zeros((2, 6))
        fitness = 0
        
        alphas = forward(observations, transition_matrix, emission_matrix, pi)
        betas = backward(observations, transition_matrix, emission_matrix)
        
        #collect probabilities of each observation
        probability_of_obs = pi[0] * betas[0][0] + pi[1] * betas[0][1]
        fitness += probability_of_obs
        
        #xi calculations - probability of being in state 'i' at time 't' 
        #and state 'j' at time 't+1' based on the model
        xi = np.zeros([observations_rows, K, K])
        for t in range(observations_rows - 1):
            denominator = np.dot(np.dot(alphas[t, :].T, transition_matrix) * emission_matrix[:, observations[t + 1]].T, betas[t + 1, :])
            xi[t, 0, 0] = (alphas[t][0] * transition_matrix[0, 0]
                          * emission_matrix[0][observations[t + 1]] * betas[t + 1][0]) / denominator
            xi[t, 0, 1] = (alphas[t][0] * transition_matrix[0, 1]
                          * emission_matrix[1][observations[t + 1]] * betas[t + 1][1]) / denominator
            xi[t, 1, 0] = (alphas[t][1] * transition_matrix[1, 0]
                          * emission_matrix[0][observations[t + 1]] * betas[t + 1][0]) / denominator
            xi[t, 1, 1] = (alphas[t][1] * transition_matrix[1, 1]
                          * emission_matrix[1][observations[t + 1]] * betas[t + 1][1]) / denominator
        
        #gamma stores probability that we are in state 'i' at time t
        gamma = np.zeros([observations_rows, K])
        for t in range(observations_rows - 1):
            gamma[t, 0] = np.sum(xi[t, 0, :])
            gamma[t, 1] = np.sum(xi[t, 1, :])
        
        #recompute initial probabilities for the observation
        new_pi = gamma[0] / np.sum(gamma[0])
        
        # calculate new transition matrix for later update
        new_transition_matrix[0, 0] += np.sum(xi[:, 0, 0]) / np.sum(gamma[:, 0])
        new_transition_matrix[0, 1] += np.sum(xi[:, 0, 1]) / np.sum(gamma[:, 0])
        new_transition_matrix[1, 0] += np.sum(xi[:, 1, 0]) / np.sum(gamma[:, 1])
        new_transition_matrix[1, 1] += np.sum(xi[:, 1, 1]) / np.sum(gamma[:, 1])
        
        # calculate new emission matrix for later update
        emit_denom = np.sum(gamma, axis=0)
        for l in range(emission_matrix.shape[1]):
            #new_emission_matrix[:, l] = np.sum(gamma[observations == l, :]) / emit_denom
            new_emission_matrix[0, l] = np.sum(np.take(gamma[:, 0], np.where(observations == l))) / emit_denom[0]
            new_emission_matrix[1, l] = np.sum(np.take(gamma[:, 1], np.where(observations == l))) / emit_denom[1]
        
        #update all values for probabilities for next epoch
        pi = new_pi
        transition_matrix = new_transition_matrix
        emission_matrix = new_emission_matrix
        
                                                                    
    # Some logging is always good :)
        print('= EPOCH #{} ='.format(epoch))
        print('Transition Matrix:', transition_matrix)
        print('Initial Dice Probability:', pi)
        print('Emission Matrix:', emission_matrix)
        print('Fitness:', fitness)
        print()

In [1183]:
baum_welch(obs_b, trans_EM, emit_EM, pi_EM, epochs = 1000)

= EPOCH #0 =
Transition Matrix: [[0.71655637 0.28344363]
 [0.69781136 0.30218864]]
Initial Dice Probability: [0.3346804 0.6653196]
Emission Matrix: [[0.22717641 0.21679365 0.24130621 0.07142966 0.15794935 0.08534472]
 [0.13603955 0.1725865  0.08334859 0.44244932 0.05669779 0.10887826]]
Fitness: 0.5005978424328097

= EPOCH #1 =
Transition Matrix: [[0.70637389 0.29362611]
 [0.69042981 0.30957019]]
Initial Dice Probability: [0.57419195 0.42580805]
Emission Matrix: [[0.17291581 0.22600451 0.19559263 0.24355826 0.04646592 0.11546287]
 [0.24594632 0.15242039 0.17389019 0.09586813 0.28468624 0.04718873]]
Fitness: 0.4987383208560826

= EPOCH #2 =
Transition Matrix: [[0.70293446 0.29706554]
 [0.68594638 0.31405362]]
Initial Dice Probability: [0.18214002 0.81785998]
Emission Matrix: [[0.2413453  0.17003675 0.20580671 0.19681334 0.15294686 0.03305104]
 [0.1084101  0.26531219 0.15430794 0.19266136 0.06705197 0.21225644]]
Fitness: 0.5000646485009852

= EPOCH #3 =
Transition Matrix: [[0.69879331 0.3

= EPOCH #26 =
Transition Matrix: [[0.61571562 0.38428438]
 [0.59463524 0.40536476]]
Initial Dice Probability: [0.42741257 0.57258743]
Emission Matrix: [[0.23290153 0.1752756  0.1998306  0.18926073 0.14818107 0.05455046]
 [0.14600566 0.23950655 0.17266383 0.20405199 0.09037677 0.1473952 ]]
Fitness: 0.49993285990034675

= EPOCH #27 =
Transition Matrix: [[0.6144934  0.3855066 ]
 [0.59271335 0.40728665]]
Initial Dice Probability: [0.5499881 0.4500119]
Emission Matrix: [[0.12090998 0.24913841 0.17188033 0.21666946 0.12542113 0.1159807 ]
 [0.29326694 0.14201291 0.20970881 0.16835676 0.1224457  0.06420888]]
Fitness: 0.4998835778658395

= EPOCH #28 =
Transition Matrix: [[0.61214466 0.38785534]
 [0.5893726  0.4106274 ]]
Initial Dice Probability: [0.55713915 0.44286085]
Emission Matrix: [[0.24059464 0.12605856 0.22802359 0.17716571 0.13496548 0.09319202]
 [0.13605042 0.30705813 0.13389266 0.2206605  0.10910505 0.09323324]]
Fitness: 0.49987879259254236

= EPOCH #29 =
Transition Matrix: [[0.610827

= EPOCH #52 =
Transition Matrix: [[0.58698551 0.41301449]
 [0.56142156 0.43857844]]
Initial Dice Probability: [0.60194095 0.39805905]
Emission Matrix: [[0.23359877 0.14266996 0.22071166 0.17880698 0.13284586 0.09136677]
 [0.15005846 0.277113   0.14778696 0.21643856 0.11306242 0.0955406 ]]
Fitness: 0.4997506673423988

= EPOCH #53 =
Transition Matrix: [[0.58646429 0.41353571]
 [0.56075784 0.43924216]]
Initial Dice Probability: [0.63879797 0.36120203]
Emission Matrix: [[0.19059365 0.23802349 0.13198794 0.22717858 0.11296618 0.09925016]
 [0.20442999 0.15670587 0.25966959 0.1554471  0.13814124 0.08560622]]
Fitness: 0.5005136843862636

= EPOCH #54 =
Transition Matrix: [[0.58558221 0.41441779]
 [0.56007139 0.43992861]]
Initial Dice Probability: [0.59346357 0.40653643]
Emission Matrix: [[0.21129094 0.19730513 0.22344141 0.13797852 0.14470089 0.0852831 ]
 [0.17884171 0.20780527 0.14567123 0.2658958  0.09885174 0.10293425]]
Fitness: 0.49877517647555747

= EPOCH #55 =
Transition Matrix: [[0.58473

= EPOCH #78 =
Transition Matrix: [[0.57329479 0.42670521]
 [0.54546654 0.45453346]]
Initial Dice Probability: [0.61398851 0.38601149]
Emission Matrix: [[0.20988391 0.19532346 0.21963056 0.14725576 0.14196416 0.08594215]
 [0.18119664 0.20991688 0.15183247 0.25221341 0.10306195 0.10177864]]
Fitness: 0.49879831620332415

= EPOCH #79 =
Transition Matrix: [[0.57280693 0.42719307]
 [0.54521882 0.45478118]]
Initial Dice Probability: [0.68338393 0.31661607]
Emission Matrix: [[0.1797288  0.21383266 0.1810669  0.22580796 0.09371357 0.1058501 ]
 [0.21706115 0.18787832 0.19744033 0.15903851 0.16050592 0.07807577]]
Fitness: 0.5005724270986114

= EPOCH #80 =
Transition Matrix: [[0.57265079 0.42734921]
 [0.54500642 0.45499358]]
Initial Dice Probability: [0.55984448 0.44015552]
Emission Matrix: [[0.22213896 0.18269607 0.19760638 0.18588768 0.14207291 0.06959801]
 [0.16624128 0.22519034 0.17762267 0.2068485  0.1025773  0.1215199 ]]
Fitness: 0.49976722275446905

= EPOCH #81 =
Transition Matrix: [[0.5721

= EPOCH #104 =
Transition Matrix: [[0.56522246 0.43477754]
 [0.53563872 0.46436128]]
Initial Dice Probability: [0.58224287 0.41775713]
Emission Matrix: [[0.21981535 0.18421726 0.19747119 0.1850664  0.140772   0.0726578 ]
 [0.16976758 0.2227958  0.17807168 0.20750141 0.10467107 0.11719245]]
Fitness: 0.49971567803388917

= EPOCH #105 =
Transition Matrix: [[0.56481714 0.43518286]
 [0.53500881 0.46499119]]
Initial Dice Probability: [0.65138139 0.34861861]
Emission Matrix: [[0.15629996 0.23038264 0.17593759 0.20926356 0.12012369 0.10799256]
 [0.24199523 0.17025043 0.20261157 0.17991236 0.12858032 0.07665009]]
Fitness: 0.5001335483312132

= EPOCH #106 =
Transition Matrix: [[0.56464705 0.43535295]
 [0.53454456 0.46545544]]
Initial Dice Probability: [0.63672398 0.36327602]
Emission Matrix: [[0.22591462 0.15989135 0.21263751 0.1808957  0.13122584 0.08943498]
 [0.16258214 0.25128129 0.16031701 0.2124022  0.11579471 0.09762264]]
Fitness: 0.4996506224201356

= EPOCH #107 =
Transition Matrix: [[0.5

...

= EPOCH #884 =
Transition Matrix: [[0.53673872 0.46326128]
 [0.47971708 0.52028292]]
Initial Dice Probability: [0.63819181 0.36180819]
Emission Matrix: [[0.21033782 0.18787268 0.20135849 0.18096831 0.1330985  0.0863642 ]
 [0.18272224 0.21656176 0.17532161 0.21027326 0.11487604 0.10024509]]
Fitness: 0.4992655623097245

= EPOCH #885 =
Transition Matrix: [[0.53666345 0.46333655]
 [0.47961776 0.52038224]]
Initial Dice Probability: [0.66965698 0.33034302]
Emission Matrix: [[0.18243657 0.21654475 0.17586046 0.2093337  0.11548481 0.10033971]
 [0.21130239 0.18719098 0.20144277 0.18121351 0.13292099 0.08592936]]
Fitness: 0.5005733803695765

= EPOCH #886 =
Transition Matrix: [[0.53675067 0.46324933]
 [0.47967206 0.52032794]]
Initial Dice Probability: [0.63963468 0.36036532]
Emission Matrix: [[0.21141636 0.18684079 0.20136123 0.18167966 0.13228152 0.08642044]
 [0.1815936  0.2176431  0.17530127 0.20956085 0.11570463 0.10019654]]
Fitness: 0.4993051867280067

= EPOCH #887 =
Transition Matrix: [[0.53

= EPOCH #910 =
Transition Matrix: [[0.53677916 0.46322084]
 [0.47904938 0.52095062]]
Initial Dice Probability: [0.63836603 0.36163397]
Emission Matrix: [[0.21134437 0.18690623 0.20136585 0.18165054 0.1323049  0.08642811]
 [0.18168104 0.21756195 0.17530824 0.20957826 0.11568804 0.10018247]]
Fitness: 0.4993005191971941

= EPOCH #911 =
Transition Matrix: [[0.53671165 0.46328835]
 [0.47895116 0.52104884]]
Initial Dice Probability: [0.66701413 0.33298587]
Emission Matrix: [[0.18254165 0.21756322 0.1749228  0.20933233 0.11590046 0.09973954]
 [0.21118585 0.18616128 0.20239124 0.18122398 0.13249127 0.08654638]]
Fitness: 0.5005844446605887

= EPOCH #912 =
Transition Matrix: [[0.53677266 0.46322734]
 [0.47898225 0.52101775]]
Initial Dice Probability: [0.6384809 0.3615191]
Emission Matrix: [[0.21032938 0.18711948 0.20248538 0.18087683 0.13238546 0.08680346]
 [0.18274067 0.21732525 0.17417335 0.21035686 0.11561488 0.099789  ]]
Fitness: 0.49925885620379457

= EPOCH #913 =
Transition Matrix: [[0.536

= EPOCH #936 =
Transition Matrix: [[0.53681629 0.46318371]
 [0.4783768  0.5216232 ]]
Initial Dice Probability: [0.63720823 0.36279177]
Emission Matrix: [[0.21035604 0.18712939 0.20241543 0.18092689 0.13238505 0.0867872 ]
 [0.18272361 0.21730379 0.17425572 0.21029447 0.11562156 0.09980085]]
Fitness: 0.4992585638710566

= EPOCH #937 =
Transition Matrix: [[0.53676262 0.46323738]
 [0.47830108 0.52169892]]
Initial Dice Probability: [0.66603921 0.33396079]
Emission Matrix: [[0.18319463 0.21639842 0.17502043 0.21027468 0.11537921 0.09973263]
 [0.21051955 0.18734982 0.20229157 0.18026254 0.13302307 0.08655345]]
Fitness: 0.5006294114124744

= EPOCH #938 =
Transition Matrix: [[0.53681992 0.46318008]
 [0.47833409 0.52166591]]
Initial Dice Probability: [0.6356562 0.3643438]
Emission Matrix: [[0.21034436 0.1878021  0.20143099 0.18100031 0.13299727 0.08642497]
 [0.18273973 0.21660903 0.17527004 0.21021474 0.11499589 0.10017057]]
Fitness: 0.4992608079283042

= EPOCH #939 =
Transition Matrix: [[0.5367

= EPOCH #962 =
Transition Matrix: [[0.5368769  0.4631231 ]
 [0.47774276 0.52225724]]
Initial Dice Probability: [0.63448936 0.36551064]
Emission Matrix: [[0.21035017 0.18777225 0.20145917 0.18101354 0.13295822 0.08644666]
 [0.18274368 0.2166293  0.17525053 0.2101907  0.11504239 0.1001434 ]]
Fitness: 0.4992589244189186

= EPOCH #963 =
Transition Matrix: [[0.53680391 0.46319609]
 [0.47764593 0.52235407]]
Initial Dice Probability: [0.66549656 0.33450344]
Emission Matrix: [[0.18258936 0.21653142 0.17574224 0.20941307 0.11550075 0.10022316]
 [0.21111287 0.18723901 0.20153317 0.18116567 0.13288429 0.086065  ]]
Fitness: 0.5005833747358609

= EPOCH #964 =
Transition Matrix: [[0.53689108 0.46310892]
 [0.47770168 0.52229832]]
Initial Dice Probability: [0.63547121 0.36452879]
Emission Matrix: [[0.21120461 0.18702697 0.20138321 0.18158904 0.13235161 0.08644456]
 [0.18185209 0.2174095  0.17531466 0.2096154  0.11565556 0.10015279]]
Fitness: 0.499290893036734

= EPOCH #965 =
Transition Matrix: [[0.536

= EPOCH #988 =
Transition Matrix: [[0.5369606  0.4630394 ]
 [0.47712449 0.52287551]]
Initial Dice Probability: [0.63416938 0.36583062]
Emission Matrix: [[0.21115144 0.18707034 0.20139343 0.18156339 0.13237006 0.08645135]
 [0.18191778 0.21735347 0.17531409 0.20963106 0.11564299 0.10014061]]
Fitness: 0.499286983493747

= EPOCH #989 =
Transition Matrix: [[0.53689159 0.46310841]
 [0.47702614 0.52297386]]
Initial Dice Probability: [0.66310049 0.33689951]
Emission Matrix: [[0.18258837 0.21735775 0.17507256 0.20935274 0.1158436  0.09978498]
 [0.21110442 0.18640757 0.20220631 0.18123644 0.13252953 0.08651573]]
Fitness: 0.5005884829660167

= EPOCH #990 =
Transition Matrix: [[0.53695955 0.46304045]
 [0.47706435 0.52293565]]
Initial Dice Probability: [0.63429878 0.36570122]
Emission Matrix: [[0.21040955 0.1871509  0.20227965 0.18101771 0.13238908 0.08675311]
 [0.18269029 0.21725825 0.1744168  0.21017853 0.11563049 0.09982564]]
Fitness: 0.49925745312526665

= EPOCH #991 =
Transition Matrix: [[0.53